In [6]:
import os
import openai
from dotenv import load_dotenv


openai_api_key = os.getenv("OPENAI_API_KEY") 
openai.api_key = openai_api_key

# Search
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Using LlamaIndex as a Callable Tool

In [13]:
import logging

from langchain.agents import AgentExecutor, AgentType, Tool, initialize_agent
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# logging.basicConfig()
# logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

qa_template = Prompt(
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Do not give me an answer if it is not mentioned in the context as a fact. \n"
    "Always state where the information was found, and if possible where more information can be accessed."
    "Given this information, please provide me with an answer to the following question:\n{query_str}\n"
)

# LLM
llm = ChatOpenAI(temperature=0)

## Index tool from documents in vector store
storage_context = StorageContext.from_defaults(persist_dir="vector_db")
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine(
    streaming=True, text_qa_template=qa_template, similarity_top_k=5
)

## Websearch Tool
# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
)

# Search
search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search,
)

# chain
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)

# qa_chain({"question": q})


tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(query_engine.query(q)),
        description="useful for when you want to answer questions about work and unemployment. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
    Tool(
        name="Websearch",
        func=lambda q: str(qa_chain({"question": q})),
        description="useful for when you want to answer questions about work and unemployment and the LlamaIndex Tool does not have a sufficient answer in it's documents. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")

agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    #   memory=memory
)

agent_executor.run(input="Where can I find information about the duties of landlords?")

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
file_cache is only supported with oauth2client<4.0.0
file_cache is only supported with oauth2client<4.0.0
file_cache is only supported with oauth2client<4.0.0
file_cache is only supported with oauth2client<4.0.0
file_cache is only supported with oauth2client<4.0.0
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200

'Based on the provided context information, there is no specific mention of information about the duties of landlords. Therefore, it is not possible to provide an answer to your question based on the given information.'

# Agent with Tools - openai cookbook
https://cookbook.openai.com/examples/how_to_build_a_tool-using_agent_with_langchain

In [19]:
from typing import List, Union
import re
import openai
import pandas as pd
from langchain import LLMChain, OpenAI, SerpAPIWrapper

# Langchain imports
from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
)
from langchain.chains import RetrievalQA

# LLM wrapper
from langchain.chat_models import ChatOpenAI

# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings

# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
from langchain.vectorstores import Pinecone
from tqdm.auto import tqdm

## template with history

In [14]:
# Set up a prompt template which can interpolate the history
template_with_history = """You are BureauBot, a professional search engine who provides informative answers to users. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give detailed, informative answers

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

## template without history

In [26]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [27]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [28]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [29]:
llm = OpenAI(temperature=0)

## Index tool from documents in vector store
storage_context = StorageContext.from_defaults(persist_dir="vector_db")
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine(
    streaming=True, 
    text_qa_template=qa_template, 
    similarity_top_k=5
)

tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(query_engine.query(q)),
        description="useful for when you want to answer questions about work and unemployment. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
    Tool(
        name="Websearch",
        func=lambda q: str(qa_chain({"question": q})),
        description="useful for when you want to answer questions about work and unemployment and the LlamaIndex Tool does not have a sufficient answer in it's documents. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

# Re-initialize the agent with our new list of tools
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
multi_tool_names = [tool.name for tool in tools]
multi_tool_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=multi_tool_names
)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [30]:
multi_tool_memory = ConversationBufferWindowMemory(k=2)

multi_tool_executor = AgentExecutor.from_agent_and_tools(
    agent=multi_tool_agent, tools=tools, verbose=True, memory=multi_tool_memory
)

In [31]:
multi_tool_executor.run("Where can I find information about the duties of landlords?")



> Entering new AgentExecutor chain...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"

Thought: I should look for information about the duties of landlords
Action: Websearch
Action Input: duties of landlordsINFO:langchain.retrievers.web_research:Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4295 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}